In [2]:
from langchain_core.prompts import PromptTemplate
# from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool, AgentExecutor, AgentType

# Initialize the model 'Conversational' variant of GPT-4o
# llm = ChatOpenAI(temperature=0.9, model_name="gpt-4o")
# Initialize the model with Ollama
llm = ChatOllama(model="gemma:2b", temperature=0.9)

# Initialize SerpAPI tool using wrapper
serpapi_wrapper = SerpAPIWrapper()

serpapi_tool = Tool(
    name="serpapi",
    description="busca informacion en la web usando una tool -serpapi-.",
    func=serpapi_wrapper.run
)

# Create an agent with SerpAPI tool
agent = initialize_agent(
    tools=[serpapi_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True
)

# First chain: Agent generates text using SerpAPI
story_prompt = PromptTemplate.from_template(
    "Usa la herramienta de busqueda para buscar un texto de Borges en la web (de 3 a 5 lineas) relacionadas con alguno de sus mas famosos textos."
)
# story_chain = story_prompt | agent | StrOutputParser()
story_chain = story_prompt | agent | (lambda x: x['output']) | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template(
    "Analiza en sentimiento de la historia:\n{story}"
)
analysis_chain = analysis_prompt | llm | StrOutputParser()

# Third chain: Rewrite the story with a specified sentiment
rewrite_prompt = PromptTemplate.from_template(
    "Reescribe la siguiente historia con un sentimiento {sentiment}:\n{story}"
)
rewrite_chain = rewrite_prompt | llm | StrOutputParser()

# Combine chains
# story_with_analysis = story_chain | analysis_chain
# Run the combined chain
# story_analysis = story_with_analysis.invoke({"topic": "a rainy day"})
# print("\nAnalysis:", story_analysis)

# Using RunnablePassthrough.assign to preserve original data
enhanced_chain = RunnablePassthrough.assign(
    story=story_chain  # Add 'story' key with generated content
).assign(
    analysis=analysis_chain  # Add 'analysis' key with analysis of the story
).assign(
    rewritten_story=lambda x: rewrite_chain.invoke({"story": x["story"], "sentiment": "humoristico"})
)

# Execute the chain
result = enhanced_chain.invoke({"topic": "El azar"})
print("Story:", result['story'])
print("\nAnalysis:", result['analysis'])
print("\nRewritten Story (Happy):", result['rewritten_story'])
print(result.keys())



Story: ** La herramienta de busqueda no encontró ningún texto de Borges en la web que coincida con los criterios especificados.

Analysis: **Sentimiento:**

El sentimiento de la historia es de frustración y curiosidad. 

* **Frustración:** El lector está frustrado al no encontrar ninguna referencia al texto de Borges en la web.
* **Curiosidad:** El lector está interesado en encontrar un texto de Borges y descubrir más sobre él.

**Otros sentimientos que podrían existir:**

* **Resentimiento:** El lector puede sentir un poco de ressentimiento al no encontrar lo que busca.
* **Discepción:** El lector puede estartuyando un texto de Borges que no estaba buscando.
* **Satisfacción:** El lector puede sentir satisfacción al encontrar un texto de Borges que lo interesa.

Rewritten Story (Happy): La herramienta de búsqueda se quedó cansada, como un niño que no tiene mucha energía. No encontró ningún texto de Borges en la web que coincida con los criterios especificados. "¡Por supuesto!", pensó,